# Gini 네비게이션의 InputHistory.dat파일 분석

+ Gini 네비게이션에서 통합검색 기능을 사용한 사용자의 검색 기록이 저장되는 InputHistory.dat 파일을 분석함.
+ InputHistroy 파일은 UserData에 존재.

In [41]:
import struct
import time

In [42]:
Last_Data = open('C:/Users/LEEKEONYONG/Desktop/navigation_backup/UserData/InputHistory.dat', 'rb')
data = Last_Data.read()
Last_Data.close()

### 메소드 설명
+ Gini 네비게이션 상의 InputHistory.dat 파일의 데이터 분석을 위한 함수 정의
+ 인용한 논문에 따르면, offset 0부터 1바이트는 총 데이터의 갯수, 8부터 1바이트는 데이터 순서, 13부터 1바이트는 검색한 장소의 문자수, 14부터 13의 내용만큼의 바이트는 검색한 장소의 이름을 표현함
     + 입력 시간을 가리키는 offset 16부터 4바이트는 사용자의 검색 시간을 나타냄.
     + 따라서, strcut 라이브러리의 unpack 메소드를 이용하여 '<' 파라미터를 통해리틀 앤디언 형식의 bytearray를 10진수로 변환함.
          
     ```python
    Search_Time = struct.unpack('<HH',data[i:i+4])
    Search_Time = (Search_Time[1]<<16)+Search_Time[0]
     ```
     
     + 위의 식에서 반환 된 데이터는 UTC(시)로 기준시인 1970년 1월 1일에 초단위로 더하여야함.
     + 이는 time라이브러리의 gmtime 메소드를 이용하여 표준시를 구함.
     
     ```python
    Search_Time = time.gmtime(Search_Time)
     ```
     
     + 검색된 단어(목적지)등의 목록의 글자수는 offset 13부터 1바이트에 나타나 있음.
     + 검색된 단어(목적지)등은 offset 14바이트부터 offset 13에 나타나 있는 데이터의 2배 길이만큼(한글은 2바이트로 표현) 나타나있음.
     + 검색된 단어(목적지)는 utf-16형식으로 디코딩 하면 그 내용을 볼 수 있음.
     
     ```python
    Search_place = data[i+41+1:i+41+1+data[i+41]*2].decode('utf-16')
     ```

In [43]:
def GiniInputHistorydat(data):
    Search_Data = []
    Offset_in_InputHistorybin = 68
    Data_All = data[0]
    
    for i in range(9,Offset_in_InputHistorybin*data[0],Offset_in_InputHistorybin):
        Search_Time = struct.unpack('<HH',data[i:i+4])
        Search_Time = (Search_Time[1]<<16)+Search_Time[0]
        Search_Time = time.gmtime(Search_Time)
        Search_Time = str(Search_Time[0])+'년 '+str(Search_Time[1])+'월 '+str(Search_Time[2])+'일 '+str(Search_Time[3])+'시 '+str(Search_Time[4])+'분'
        
        Search_place = data[i+5:i+5+data[i+4]*2].decode('utf-16')
        Search_Data.append((Search_Time,Search_place))
        
    return [Data_All,Search_Data]

In [44]:
GiniInputHistory = GiniInputHistorydat(data)

In [45]:
print('총 데이터의 개수: '+str(GiniInputHistory[0]))

총 데이터의 개수: 6


In [46]:
for i in range(len(GiniInputHistory[1])):
    print(str(i+1)+'번째 검색 단어 ' + str(GiniInputHistory[1][i][1])+'의 검색시간은 '+str(GiniInputHistory[1][i][0])+ '입니다' )

1번째 검색 단어 충주터미널의 검색시간은 2019년 8월 9일 12시 42분입니다
2번째 검색 단어 충주터미널의 검색시간은 2019년 12월 19일 12시 46분입니다
3번째 검색 단어 충주삼성디지털의 검색시간은 2019년 12월 25일 12시 58분입니다
4번째 검색 단어 삼성디지털의 검색시간은 2019년 12월 25일 12시 59분입니다
5번째 검색 단어 충주삼성디지털의 검색시간은 2019년 12월 25일 12시 59분입니다
6번째 검색 단어 다이소의 검색시간은 2020년 1월 27일 10시 54분입니다
